In [3]:
import os

In [4]:
%pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC'

config.yaml is updated now updating the entity

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path  # Added validation data path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

add configuration manager

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2025-03-28 13:18:36.211807: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 13:18:36.369197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-28 13:18:36.369236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-28 13:18:36.392685: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 13:18:36.441708: I tensorflow/core/platform/cpu_feature_guar

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data/train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data/valid")  # Fixed validation path

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),  # Include validation data
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

updating the component

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None 

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Reset optimizer
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        # self.valid_generator = valid_datagenerator.flow_from_directory(
        #     directory=self.config.validation_data,
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     class_mode="categorical",  # Fixed: Ensures multi-class classification
        #     shuffle=False
        # )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            **dataflow_kwargs,  # UPDATED: Used **dataflow_kwargs to simplify
            class_mode="categorical",  
            shuffle=False
        )

        # Training data generator
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        # self.train_generator = train_datagenerator.flow_from_directory(
        #     directory=self.config.training_data,
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     class_mode="categorical",  # Fixed: Ensures correct class label handling
        #     shuffle=True
        # )

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            **dataflow_kwargs,  # UPDATED: Used **dataflow_kwargs
            class_mode="categorical",
            shuffle=True
        )
        print("Train Class Indices:", self.train_generator.class_indices)
        print("Train Num Classes:", self.train_generator.num_classes)
        print("Valid Class Indices:", self.valid_generator.class_indices)
        print("Valid Num Classes:", self.valid_generator.num_classes)

    
    def save_model(self,path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        if not hasattr(self.model, "optimizer"):
            self.model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Reset optimizer
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        


In [12]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


update the pipeline


In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2025-03-28 13:18:38,109: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-28 13:18:38,111: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-28 13:18:38,112: INFO: common: created directory at: artifacts]
[2025-03-28 13:18:38,112: INFO: common: created directory at: artifacts/training]


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']